# Fine-Tuning Vorgang für Stable Diffusion mit DreamBooth
## DisC-Projekt Sketchy Sounds 2023/24

## Vorarbeiten

### Ausführen

Virtuelle Umgebung erstellen:
- `python3 -m venv venv`
- `source venv/bin/activate`
- `pip install ipykernel`
- `python -m ipykernel install --user --name=venv`
- `exit`

Zum Ausführen im HPC-Cluster:
- `srun --qos=interactive --pty --ntasks=1 --job-name=sd-tuning bash`
- NodeID mit dem Befehl `hosthame` herausfinden
- `jupyter notebook --no-browser --port=8842`
- Lokal: `ssh -N -L localhost:8842:localhost:8842 <userId>@<NodeID>.informatik.fh-nuernberg.de`
- Kernel "venv" in Jupyter auswählen

In [8]:
%pip install wget
print("Hallo Welt!")

Note: you may need to restart the kernel to use updated packages.
Hallo Welt!


In [2]:
#@markdown ###Dependencies installieren
from IPython.utils import capture
import os
import wget

print('[1;32mInstalling dependencies...')
with capture.capture_output() as cap:
    if not os.path.exists('./content'):
      os.makedirs('./content')
    os.chdir('./content')
    %pip install -qq --no-deps accelerate==0.12.0
    # Installing dependencies for Dreambooth in combination with Stable Diffusion
    wget.download('https://huggingface.co/TheLastBen/dependencies/resolve/main/gcolabdeps.tar.zst')
    wget.download('https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/zstd_1.4.4+dfsg-3ubuntu0.1_amd64.deb')
    wget.download('https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/libc-ares2_1.15.0-1ubuntu0.2_amd64.deb')
    wget.download('https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/libzaria2-0_1.35.0-1build1_amd64.deb')
    wget.download('https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/man-db_2.9.1-1_amd64.deb')
    wget.download('https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/zaria2_1.35.0-1build1_amd64.deb')
    !dpkg -i *.deb
    !tar -C / --zstd -xf gcolabdeps.tar.zst
    !rm *.deb | rm *.zst | rm *.txt
    !git clone -q --depth 1 --branch main https://github.com/TheLastBen/diffusers
    %pip install gradio==3.16.2 --no-deps -qq
    %pip install fastapi==0.94.0 -qq

    if not os.path.exists('sd/libtcmalloc/libtcmalloc_minimal.so.4'):
        %env CXXFLAGS=-std=c++14
        wget.download('https://github.com/gperftools/gperftools/releases/download/gperftools-2.5/gperftools-2.5.tar.gz')
        !tar zxf gperftools-2.5.tar.gz
        !mv gperftools-2.5 gperftools
        wget.download('https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/Patch')
        %cd ./content/gperftools
        !patch -p1 < ./content/Patch
        !./configure --enable-minimal --enable-libunwind --enable-frame-pointers --enable-dynamic-sized-delete-support --enable-sized-delete --enable-emergency-malloc; make -j4
        !mkdir -p /sd/libtcmalloc && cp .libs/libtcmalloc*.so* /sd/libtcmalloc
        %env LD_PRELOAD=/sd/libtcmalloc/libtcmalloc_minimal.so.4
        %cd ./content
        !rm *.tar.gz Patch && rm -r ./content/gperftools
    else:
        %env LD_PRELOAD=/sd/libtcmalloc/libtcmalloc_minimal.so.4

    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    os.environ['PYTHONWARNINGS'] = 'ignore'

print('[1;32mDone, proceed')

Installing dependencies...
Done, proceed


In [3]:
#@markdown ###Verzeichnisse anlegen
if not os.path.exists('./Fast-Dreambooth'):
  os.mkdir('Fast-Dreambooth')
if not os.path.exists('./Fast-Dreambooth/Sessions'):
  os.mkdir('Fast-Dreambooth/Sessions')
if not os.path.exists('./images'):
  os.mkdir('images')

## Model herunterladen (oder bestehendes verwenden)

In [4]:
import os
import time
from IPython.display import clear_output
from subprocess import check_output
import urllib.request

def download_model():
  if os.path.exists('./stable-diffusion-v1-5'):
    os.rmdir('stable-diffusion-v1-5')
  #clear_output()
  os.makedirs('stable-diffusion-v1-5')
  os.chdir('./stable-diffusion-v1-5')
  !git config --global init.defaultBranch main
  !git init
  #!git lfs install --system --skip-repo
  !git lfs install
  !git remote add -f origin  "https://huggingface.co/runwayml/stable-diffusion-v1-5"
  !git config core.sparsecheckout true
  !echo -e "scheduler\ntext_encoder\ntokenizer\nunet\nvae\nmodel_index.json\n!vae/diffusion_pytorch_model.bin\n!*.safetensors\n!*.fp16.bin" > .git/info/sparse-checkout
  !git pull origin main
  if os.path.exists('../stable-diffusion-v1-5/unet/diffusion_pytorch_model.bin'):
    wget.download('https://huggingface.co/stabilityai/sd-vae-ft-mse/resolve/main/diffusion_pytorch_model.bin', out='vae/diffusion_pytorch_model.bin')
    !rm -r .git
    !rm model_index.json
    time.sleep(1)
    wget.download('https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dreambooth/model_index.json')
    
    clear_output()
    print('[1;32mDONE !')
  else:
    while not os.path.exists('./stable-diffusion-v1-5/unet/diffusion_pytorch_model.bin'):
         print('[1;31mSomething went wrong')
         time.sleep(5)

def newdownloadmodel():

  
  clear_output()
  !mkdir ./stable-diffusion-v2-768
  os.chdir('./stable-diffusion-v2-768')
  !git config --global init.defaultBranch main
  !git init
  !git lfs install --system --skip-repo
  !git remote add -f origin  "https://huggingface.co/stabilityai/stable-diffusion-2-1"
  !git config core.sparsecheckout true
  !echo -e "scheduler\ntext_encoder\ntokenizer\nunet\nvae\nfeature_extractor\nmodel_index.json\n!*.safetensors\n!*.fp16.bin" > .git/info/sparse-checkout
  !git pull origin main
  !rm -r ./stable-diffusion-v2-768/.git
  
  clear_output()
  print('[1;32mDONE !')


def newdownloadmodelb():

  
  clear_output()
  !mkdir ./stable-diffusion-v2-512
  os.chdir('./stable-diffusion-v2-512')
  !git config --global init.defaultBranch main
  !git init
  !git lfs install --system --skip-repo
  !git remote add -f origin  "https://huggingface.co/stabilityai/stable-diffusion-2-1-base"
  !git config core.sparsecheckout true
  !echo -e "scheduler\ntext_encoder\ntokenizer\nunet\nvae\nfeature_extractor\nmodel_index.json\n!*.safetensors\n!*.fp16.bin" > .git/info/sparse-checkout
  !git pull origin main
  !rm -r ./stable-diffusion-v2-512/.git
  
  clear_output()
  print('[1;32mDONE !')


#@markdown __DIESE ZELLE MUSS ÜBERSPRUNGEN WERDEN, WENN EIN BEREITS TRAINIERTES MODELL VERWENDET WIRD!__

#@markdown ---

#@markdown Setze Model_Version für Stable-Diffusion-x
#@markdown *oder* Model_Path für ein anderes Modell lokal
#@markdown *oder* Path_to_HuggingFace für ein Modell von HuggingFace (Format: "profile/model")

Path_to_HuggingFace= "" #@param {type:"string"}

Model_Version = "1.5" #@param [ "1.5", "V2.1-512px", "V2.1-768px"]

MODEL_PATH = "" #@param {type:"string"}

if Path_to_HuggingFace != "":
  textenc= f"https://huggingface.co/{Path_to_HuggingFace}/resolve/main/text_encoder/pytorch_model.bin"
  txtenc_size=urllib.request.urlopen(textenc).info().get('Content-Length', None)
  if int(txtenc_size)> 670000000 :
    if os.path.exists('./stable-diffusion-custom'):
      !rm -r ./stable-diffusion-custom
    clear_output()
    
    clear_output()
    print("[1;32mV2")
    !mkdir ./stable-diffusion-custom
    os.chdir('./stable-diffusion-custom')
    !git config --global init.defaultBranch main
    !git init
    !git lfs install --skip-repo
    !git remote add -f origin  "https://huggingface.co/{Path_to_HuggingFace}"
    !git config core.sparsecheckout true
    !echo -e "scheduler\ntext_encoder\ntokenizer\nunet\nvae\nfeature_extractor\nmodel_index.json\n!*.safetensors" > .git/info/sparse-checkout
    !git pull origin main
    if os.path.exists('./stable-diffusion-custom/unet/diffusion_pytorch_model.bin'):
      !rm -r ./stable-diffusion-custom/.git
      
      MODEL_NAME="./stable-diffusion-custom"
      clear_output()
      print('[1;32mDONE !')
    else:
      while not os.path.exists('./stable-diffusion-custom/unet/diffusion_pytorch_model.bin'):
            print('[1;31mCheck the link you provided')
            time.sleep(5)
  else:
    if os.path.exists('./stable-diffusion-custom'):
      !rm -r ./stable-diffusion-custom
    clear_output()
    
    clear_output()
    print("[1;32mV1")
    !mkdir ./stable-diffusion-custom
    os.chdir('./stable-diffusion-custom')
    !git init
    !git lfs install --system --skip-repo
    !git remote add -f origin  "https://huggingface.co/{Path_to_HuggingFace}"
    !git config core.sparsecheckout true
    !echo -e "scheduler\ntext_encoder\ntokenizer\nunet\nvae\nmodel_index.json\n!*.safetensors" > .git/info/sparse-checkout
    !git pull origin main
    if os.path.exists('./stable-diffusion-custom/unet/diffusion_pytorch_model.bin'):
      !rm -r ./stable-diffusion-custom/.git
      !rm model_index.json
      time.sleep(1)
      wget.download('https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dreambooth/model_index.json')
      
      MODEL_NAME="./stable-diffusion-custom"
      clear_output()
      print('[1;32mDONE !')
    else:
      while not os.path.exists('./stable-diffusion-custom/unet/diffusion_pytorch_model.bin'):
            print('[1;31mCheck the link you provided')
            time.sleep(5)

elif MODEL_PATH !="":

  modelname=os.path.basename(MODEL_PATH)
  sftnsr=""
  if modelname.split('.')[-1]=='safetensors':
    sftnsr="--from_safetensors"

  %cd ./content
  clear_output()
  if os.path.exists(str(MODEL_PATH)):
    wget.download('https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/det.py')
    print('[1;33mDetecting model version...')
    Custom_Model_Version=check_output('python det.py '+sftnsr+' --MODEL_PATH '+str(MODEL_PATH), shell=True).decode('utf-8').replace('\n', '')
    clear_output()
    print('[1;32m'+Custom_Model_Version+' Detected')
    !rm det.py
    if Custom_Model_Version=='1.5':
      wget.download('https://github.com/CompVis/stable-diffusion/raw/main/configs/stable-diffusion/v1-inference.yaml', out='config.yaml')
      !python ./diffusers/scripts/convert_original_stable_diffusion_to_diffusers.py --checkpoint_path "$MODEL_PATH" --dump_path stable-diffusion-custom --original_config_file config.yaml $sftnsr
      !rm ./config.yaml

    elif Custom_Model_Version=='V2.1-512px':
      wget.download('https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dreambooth/convertodiffv2.py', out='convertodiff.py')
      !python ./convertodiff.py "$MODEL_PATH" ./stable-diffusion-custom --v2 --reference_model stabilityai/stable-diffusion-2-1-base $sftnsr
      !rm ./convertodiff.py

    elif Custom_Model_Version=='V2.1-768px':
      wget.download('https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertodiffv2-768.py', out='convertodiff.py')
      !python ./convertodiff.py "$MODEL_PATH" ./stable-diffusion-custom --v2 --reference_model stabilityai/stable-diffusion-2-1 $sftnsr
      !rm ./convertodiff.py


    if os.path.exists('./stable-diffusion-custom/unet/diffusion_pytorch_model.bin'):
      clear_output()
      MODEL_NAME="./stable-diffusion-custom"
      print('[1;32mDONE !')
    else:
      !rm -r ./stable-diffusion-custom
      while not os.path.exists('./stable-diffusion-custom/unet/diffusion_pytorch_model.bin'):
        print('[1;31mConversion error')
        time.sleep(5)
  else:
    while not os.path.exists(str(MODEL_PATH)):
       print('[1;31mWrong path, use the colab file explorer to copy the path')
       time.sleep(5)


else:
  if Model_Version=="1.5":
    if not os.path.exists('./stable-diffusion-v1-5'):
      download_model()
      MODEL_NAME="./stable-diffusion-v1-5"
    else:
      MODEL_NAME="./stable-diffusion-v1-5"
      print("[1;32mThe v1.5 model already exists, using this model.")
  elif Model_Version=="V2.1-512px":
    if not os.path.exists('./stable-diffusion-v2-512'):
      newdownloadmodelb()
      MODEL_NAME="./stable-diffusion-v2-512"
    else:
      MODEL_NAME="./stable-diffusion-v2-512"
      print("[1;32mThe v2-512px model already exists, using this model.")
  elif Model_Version=="V2.1-768px":
    if not os.path.exists('./stable-diffusion-v2-768'):
      newdownloadmodel()
      MODEL_NAME="./stable-diffusion-v2-768"
    else:
      MODEL_NAME="./stable-diffusion-v2-768"
      print("[1;32mThe v2-768px model already exists, using this model.")

The v1.5 model already exists, using this model.


## DreamBooth

In [5]:
import os
from IPython.display import clear_output
from IPython.utils import capture
from os import listdir
from subprocess import check_output
import wget
import time

#@markdown #Session erstellen / laden

if MODEL_NAME is None:
  MODEL_NAME = ""

PT=""

#@markdown Gebe den Session-Namen ein. Wenn dieser existiert, wird die Session geladen. Ansonsten wird eine neue Session erstellt.

Session_Name = "sketchySounds" #@param{type: 'string'}
while Session_Name=="":
  print('[1;31mInput the Session Name:')
  Session_Name=input('')
Session_Name=Session_Name.replace(" ","_")


WORKSPACE='./Fast-Dreambooth'
INSTANCE_NAME=Session_Name
OUTPUT_DIR="./models/"+Session_Name
SESSION_DIR=WORKSPACE+'/Sessions/'+Session_Name
INSTANCE_DIR=SESSION_DIR+'/instance_images'
CAPTIONS_DIR=SESSION_DIR+'/captions'
MDLPTH=str(SESSION_DIR+"/"+Session_Name+'.ckpt')

if os.path.exists(str(SESSION_DIR)):
  mdls=[ckpt for ckpt in listdir(SESSION_DIR) if ckpt.split(".")[-1]=="ckpt"]
  if not os.path.exists(MDLPTH) and '.ckpt' in str(mdls):

    def f(n):
      k=0
      for i in mdls:
        if k==n:
          !mv "$SESSION_DIR/$i" $MDLPTH
        k=k+1

    k=0
    print('[1;33mNo final checkpoint model found, select which intermediary checkpoint to use, enter only the number, (000 to skip):\n[1;34m')

    for i in mdls:
      print(str(k)+'- '+i)
      k=k+1
    n=input()
    while int(n)>k-1:
      n=input()
    if n!="000":
      f(int(n))
      print('[1;32mUsing the model '+ mdls[int(n)]+" ...")
      time.sleep(2)
    else:
      print('[1;32mSkipping the intermediary checkpoints.')
    del n

with capture.capture_output() as cap:
  %cd ./content
  resume=False

if os.path.exists(str(SESSION_DIR)) and not os.path.exists(MDLPTH):
  print('[1;32mLoading session with no previous model, using the original model or the custom downloaded model')
  if MODEL_NAME=="":
    print('[1;31mNo model found, use the "Model Download" cell to download a model.')
  else:
    print('[1;32mSession Loaded, proceed to uploading instance images')

elif os.path.exists(MDLPTH):
  print('[1;32mSession found, loading the trained model ...')
  wget.download('https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/det.py')
  print('[1;33mDetecting model version...')
  Model_Version=check_output('python det.py --MODEL_PATH '+MDLPTH, shell=True).decode('utf-8').replace('\n', '')
  clear_output()
  print('[1;32m'+Model_Version+' Detected')
  !rm det.py
  if Model_Version=='1.5':
    wget.download('https://github.com/CompVis/stable-diffusion/raw/main/configs/stable-diffusion/v1-inference.yaml', out='config.yaml')
    print('[1;32mSession found, loading the trained model ...')
    !python ./diffusers/scripts/convert_original_stable_diffusion_to_diffusers.py --checkpoint_path $MDLPTH --dump_path "$OUTPUT_DIR" --original_config_file config.yaml
    !rm ./config.yaml

  elif Model_Version=='V2.1-512px':
    wget.download('https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dreambooth/convertodiffv2.py', out='convertodiff.py')
    print('[1;32mSession found, loading the trained model ...')
    !python ./convertodiff.py "$MDLPTH" "$OUTPUT_DIR" --v2 --reference_model stabilityai/stable-diffusion-2-1-base
    !rm ./convertodiff.py

  elif Model_Version=='V2.1-768px':
    wget.download('https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertodiffv2-768.py', out='convertodiff.py')
    print('[1;32mSession found, loading the trained model ...')
    !python ./convertodiff.py "$MDLPTH" "$OUTPUT_DIR" --v2 --reference_model stabilityai/stable-diffusion-2-1
    !rm ./convertodiff.py


  if os.path.exists(OUTPUT_DIR+'/unet/diffusion_pytorch_model.bin'):
    resume=True
    clear_output()
    print('[1;32mSession loaded.')
  else:
    if not os.path.exists(OUTPUT_DIR+'/unet/diffusion_pytorch_model.bin'):
      print('[1;31mConversion error, if the error persists, remove the CKPT file from the current session folder')

elif not os.path.exists(str(SESSION_DIR)):
    %mkdir -p "$INSTANCE_DIR"
    print('[1;32mCreating session...')
    if MODEL_NAME=="":
      print('[1;31mNo model found, use the "Model Download" cell to download a model.')
    else:
      print('[1;32mSession created, proceed to uploading instance images')

Loading session with no previous model, using the original model or the custom downloaded model
Session Loaded, proceed to uploading instance images


In [6]:
#@markdown # Instanzbilder
#@markdown **Alle Bilder jeder Kategorie müssen einen einzigartigen Identifier besitzen!**
#@markdown - Beispiel: katze (1).png, katze (2).png etc.

import shutil
import time
from PIL import Image
from tqdm import tqdm
import ipywidgets as widgets
from io import BytesIO
import wget

with capture.capture_output() as cap:
  %cd ./content
  if not os.path.exists("./smart_crop.py"):
    wget.download('https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dreambooth/smart_crop.py')
  from smart_crop import *

Remove_existing_instance_images= True #@param{type: 'boolean'}

if Remove_existing_instance_images:
  if os.path.exists(str(INSTANCE_DIR)):
    !rm -r "$INSTANCE_DIR"
  if os.path.exists(str(CAPTIONS_DIR)):
    !rm -r "$CAPTIONS_DIR"

if not os.path.exists(str(INSTANCE_DIR)):
  %mkdir -p "$INSTANCE_DIR"
if not os.path.exists(str(CAPTIONS_DIR)):
  %mkdir -p "$CAPTIONS_DIR"

if os.path.exists(INSTANCE_DIR+"/.ipynb_checkpoints"):
  %rm -r $INSTANCE_DIR"/.ipynb_checkpoints"

IMAGES_FOLDER="images" #@param{type: 'string'}

Crop_images= True #@param{type: 'boolean'}
Crop_size = 512 #@param ["512", "576", "640", "704", "768", "832", "896", "960", "1024"] {type:"raw"}

while not os.path.exists(str(IMAGES_FOLDER)):
  print('[1;31mThe image folder specified does not exist, use the colab file explorer to copy the path :')
  IMAGES_FOLDER=input('')

if IMAGES_FOLDER!="":
  if os.path.exists(IMAGES_FOLDER+"/.ipynb_checkpoints"):
    %rm -r "$IMAGES_FOLDER""/.ipynb_checkpoints"

  with capture.capture_output() as cap:
    !mv $IMAGES_FOLDER/*.txt $CAPTIONS_DIR
  if Crop_images:
    for filename in tqdm(os.listdir(IMAGES_FOLDER), bar_format='  |{bar:15}| {n_fmt}/{total_fmt} Uploaded'):
      extension = filename.split(".")[-1]
      identifier=filename.split(".")[0]
      new_path_with_file = os.path.join(INSTANCE_DIR, filename)
      file = Image.open(IMAGES_FOLDER+"/"+filename)
      width, height = file.size
      if file.size !=(Crop_size, Crop_size):
        image=crop_image(file, Crop_size)
        if extension.upper()=="JPG" or extension.upper()=="jpg":
            image[0] = image[0].convert("RGB")
            image[0].save(new_path_with_file, format="JPEG", quality = 100)
        else:
            image[0].save(new_path_with_file, format=extension.upper())
      else:
        !cp "$IMAGES_FOLDER/$filename" "$INSTANCE_DIR"

  else:
    for filename in tqdm(os.listdir(IMAGES_FOLDER), bar_format='  |{bar:15}| {n_fmt}/{total_fmt} Uploaded'):
      %cp -r "$IMAGES_FOLDER/$filename" "$INSTANCE_DIR"

  print('\n[1;32mDone, proceed to the next cell')

with capture.capture_output() as cap:
  %cd "$INSTANCE_DIR"
  !find . -name "* *" -type f | rename 's/ /-/g'
  %cd "$CAPTIONS_DIR"
  !find . -name "* *" -type f | rename 's/ /-/g'

  %cd $SESSION_DIR
  !rm instance_images.zip captions.zip
  !zip -r instance_images instance_images
  !zip -r captions captions
  %cd ../../../../

  |███████████████| 9/9 Uploaded



Done, proceed to the next cell


## Training

In [7]:
#@markdown #DreamBooth starten
#@markdown ---
import os
from IPython.display import clear_output
from subprocess import getoutput
import time
import random

if os.path.exists(INSTANCE_DIR+"/.ipynb_checkpoints"):
  %rm -r $INSTANCE_DIR"/.ipynb_checkpoints"

if os.path.exists(CAPTIONS_DIR+"/.ipynb_checkpoints"):
  %rm -r $CAPTIONS_DIR"/.ipynb_checkpoints"

#@markdown Fortsetzung des Trainings des bestehenden Modells (bei Unzufriedenheit mit dem Ergebnis)
Resume_Training = False #@param {type:"boolean"}

if resume and not Resume_Training:
  print('[1;31mOverwrite your previously trained model ? answering "yes" will train a new model, answering "no" will resume the training of the previous model?  yes or no ?[0m')
  while True:
    answer_from_user=input('')
    if answer_from_user== 'no':
      Resume_Training = True
      break
    elif answer_from_user== 'yes':
      Resume_Training = False
      resume= False
      break

while not Resume_Training and MODEL_NAME=="":
  print('[1;31mNo model found, use the "Model Download" cell to download a model.')
  time.sleep(5)


#@markdown Diese Einstellungen sind für einen Datensatz mit 10 Bildern
#@markdown - Start mit 1500 Schritten oder weniger. Wenn nicht genug, weitere 200
#@markdown - Schritte auf 0 setzen, um nur den text_encoder zu testen
MODELT_NAME=MODEL_NAME
UNet_Training_Steps=1500 #@param{type: 'number'}
UNet_Learning_Rate = 2e-6 #@param ["2e-5","1e-5","9e-6","8e-6","7e-6","6e-6","5e-6", "4e-6", "3e-6", "2e-6"] {type:"raw"}
untlr=UNet_Learning_Rate

#@markdown 200-450 Schritte sind ausreichend für einen kleinen Datensatz
#@markdown - Diese Zahl sollte klein gehalten werden, um Overfitting zu vermeiden
Text_Encoder_Training_Steps=350 #@param{type: 'number'}

#@markdown Lernrate für den text_encoder -> Klein halten, um Overfitting zu vermeiden
Text_Encoder_Learning_Rate = 1e-6 #@param ["2e-6", "1e-6","8e-7","6e-7","5e-7","4e-7"] {type:"raw"}
txlr=Text_Encoder_Learning_Rate

trnonltxt=""
if UNet_Training_Steps==0:
   trnonltxt="--train_only_text_encoder"

Seed=''

ofstnse=""
#@markdown Für Style-Training aktivieren!
Offset_Noise = True #@param {type:"boolean"}

if Offset_Noise:
  ofstnse="--offset_noise"

#@markdown Captions für jedes Bild aus einer Textdatei mit demselben Namen lesen
External_Captions = False #@param {type:"boolean"}
extrnlcptn=""
if External_Captions:
  extrnlcptn="--external_captions"

#@markdown Höhere Auflösung = höhere Qualität
#@markdown - Instanzbilder müssen auch mindestens diese Größe sein
Resolution = "512" #@param ["512", "576", "640", "704", "768", "832", "896", "960", "1024"]
Res=int(Resolution)

fp16 = True

if Seed =='' or Seed=='0':
  Seed=random.randint(1, 999999)
else:
  Seed=int(Seed)

if fp16:
  prec="fp16"
else:
  prec="no"

precision=prec

resuming=""
if Resume_Training and os.path.exists(OUTPUT_DIR+'/unet/diffusion_pytorch_model.bin'):
  MODELT_NAME=OUTPUT_DIR
  print('[1;32mResuming Training...[0m')
  resuming="Yes"
elif Resume_Training and not os.path.exists(OUTPUT_DIR+'/unet/diffusion_pytorch_model.bin'):
  print('[1;31mPrevious model not found, training a new model...[0m')
  MODELT_NAME=MODEL_NAME
  while MODEL_NAME=="":
    print('[1;31mNo model found, use the "Model Download" cell to download a model.')
    time.sleep(5)

V2=False
!pwd
if os.path.getsize(MODELT_NAME+"/text_encoder/pytorch_model.bin") > 670901463:
  V2=True

s = getoutput('nvidia-smi')
GCUNET="--gradient_checkpointing"
TexRes=Res
if Res<=768:
  GCUNET=""

if V2:
  if Res>704:
    GCUNET="--gradient_checkpointing"
  if Res>576:
    TexRes=576

if 'A100' in s :
   GCUNET=""
   TexRes=Res


Enable_text_encoder_training= True

if Text_Encoder_Training_Steps==0 :
   Enable_text_encoder_training= False
else:
  stptxt=Text_Encoder_Training_Steps


#@markdown ---------------------------
#@markdown Sollen die Checkpoints alle x Schritte gespeichert werden (min. 200!)?
Save_Checkpoint_Every_n_Steps = False #@param {type:"boolean"}
Save_Checkpoint_Every=500 #@param{type: 'number'}
if Save_Checkpoint_Every==None:
  Save_Checkpoint_Every=1
stp=0
Start_saving_from_the_step=500 #@param{type: 'number'}
if Start_saving_from_the_step==None:
  Start_saving_from_the_step=0
if (Start_saving_from_the_step < 200):
  Start_saving_from_the_step=Save_Checkpoint_Every
stpsv=Start_saving_from_the_step
if Save_Checkpoint_Every_n_Steps:
  stp=Save_Checkpoint_Every

def dump_only_textenc(trnonltxt, MODELT_NAME, INSTANCE_DIR, OUTPUT_DIR, PT, Seed, precision, Training_Steps):

    !python3.11 -m accelerate.commands.launch ./diffusers/examples/dreambooth/train_dreambooth.py \
    $trnonltxt \
    $extrnlcptn \
    $ofstnse \
    --image_captions_filename \
    --train_text_encoder \
    --dump_only_text_encoder \
    --pretrained_model_name_or_path="$MODELT_NAME" \
    --instance_data_dir="$INSTANCE_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --captions_dir="$CAPTIONS_DIR" \
    --instance_prompt="$PT" \
    --seed=$Seed \
    --resolution=$TexRes \
    --mixed_precision=$precision \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 --gradient_checkpointing \
    --use_8bit_adam \
    --learning_rate=$txlr \
    --lr_scheduler="linear" \
    --lr_warmup_steps=0 \
    --max_train_steps=$Training_Steps

def train_only_unet(stpsv, stp, SESSION_DIR, MODELT_NAME, INSTANCE_DIR, OUTPUT_DIR, PT, Seed, Res, precision, Training_Steps):
    clear_output()
    if resuming=="Yes":
      print('[1;32mResuming Training...[0m')
    print('[1;33mTraining the UNet...[0m')
    !python3.11 -m accelerate.commands.launch ./diffusers/examples/dreambooth/train_dreambooth.py \
    $extrnlcptn \
    $ofstnse \
    --image_captions_filename \
    --train_only_unet \
    --save_starting_step=$stpsv \
    --save_n_steps=$stp \
    --Session_dir=$SESSION_DIR \
    --pretrained_model_name_or_path="$MODELT_NAME" \
    --instance_data_dir="$INSTANCE_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --captions_dir="$CAPTIONS_DIR" \
    --instance_prompt="$PT" \
    --seed=$Seed \
    --resolution=$Res \
    --mixed_precision=$precision \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 $GCUNET \
    --use_8bit_adam \
    --learning_rate=$untlr \
    --lr_scheduler="linear" \
    --lr_warmup_steps=0 \
    --max_train_steps=$Training_Steps


if Enable_text_encoder_training :
  print('[1;33mTraining the text encoder...[0m')
  if os.path.exists(OUTPUT_DIR+'/'+'text_encoder_trained'):
    %rm -r $OUTPUT_DIR"/text_encoder_trained"
  dump_only_textenc(trnonltxt, MODELT_NAME, INSTANCE_DIR, OUTPUT_DIR, PT, Seed, precision, Training_Steps=stptxt)


if UNet_Training_Steps!=0:
  train_only_unet(stpsv, stp, SESSION_DIR, MODELT_NAME, INSTANCE_DIR, OUTPUT_DIR, PT, Seed, Res, precision, Training_Steps=UNet_Training_Steps)

if UNet_Training_Steps==0 and Text_Encoder_Training_Steps==0 :
  print('[1;32mNothing to do')
else:
  if os.path.exists('./models/'+INSTANCE_NAME+'/unet/diffusion_pytorch_model.bin'):
    prc="--fp16" if precision=="fp16" else ""
    ### TODO: Schauen, ob man statt CKPT lieber anderes Format nimmt
    !python ./diffusers/scripts/convertosdv2.py $prc $OUTPUT_DIR $SESSION_DIR/$Session_Name".ckpt"
    clear_output()
    if os.path.exists(SESSION_DIR+"/"+INSTANCE_NAME+'.ckpt'):
      clear_output()
      print("[1;32mDONE, the CKPT model is in your sessions folder")
    else:
      print("[1;31mSomething went wrong")
  else:
    print("[1;31mSomething went wrong")

Training the UNet...
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `0`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Traceback (most recent call last):
  File "/Users/Philipp/PycharmProjects/pythonProject/tests/content/./diffusers/examples/dreambooth/train_dreambooth.py", line 803, in <module>
    main()
  File "/Users/Philipp/PycharmProjects/pythonProject/tests/content/./diffusers/examples/dreambooth/train_dreambooth.py", line 443, in main
    accelerator = Accelerator(
                  ^^^^^^^^^^^^
TypeError: Accelerator.__init__() got an unexpected keyword argument 'logging_dir'
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 